# Cell Density Notebook

Ben Halligan

## Purpose

Count number of cells in given area for microscope dataset

## Images Parameters

- Size: 2000 x 2000 x 2


### TODO:

- [x] Connect to google drive
- [x] Load tiffs into dataframe
- [x] Test train split
- [x] General linear regression model
- [ ] Train
- [ ] Validate





In [1]:
import os

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms, models

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from skimage import(
    io, measure
)

# check for GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Current Device: {device}")

# Google Auth
# gauth = GoogleAuth()
# gauth.LocalWebserverAuth()

# drive = GoogleDrive(gauth)

PATH = 'data/INS1_BF/'

Current Device: cuda:0


In [3]:
file_list = os.listdir(PATH)
print(file_list)

['0', '10000', '2000', '4000', '6000', '8000']


In [4]:
# Create Data

from torchvision import datasets, transforms

data = datasets.ImageFolder(root='data/INS1_BF', transform=transforms.ToTensor())

In [ ]:
print(data.classes)

In [5]:
# Load data with data loader
from torch.utils.data import DataLoader
loader = DataLoader(data, batch_size= 64, shuffle=True)

In [ ]:
dataiter = iter(loader)
images, labels = dataiter.next()
print(type(images))
print(images.shape)
print(labels.shape)

In [ ]:
# Using VGG16 Image Model
model_vgg16 = models.vgg16(pretrained=True)

In [6]:
from typing import Tuple

def conv_layer(in_channels: int, out_channels: int, 
               kernels: Tuple[int,int], stride: int=1):
    
    layer = nn.Sequential(
         nn.Conv2d(in_channels=in_channels, out_channels=out_channels,
                     kernel_size=kernels, stride=stride, bias=False, 
                     padding=(kernels[0] //2, kernels[1] // 2)),
        nn.BatchNorm2d(num_features=out_channels),
        nn.ReLU()
    )

### Models

#### VGG16 Model Description

- Based on [this](https://github.com/LeanManager/PyTorch_Image_Classifier/blob/master/Image_Classifier_Project.ipynb) project
- [VGG16 Model](https://neurohive.io/en/popular-networks/vgg16/)

#### Density Map CNN

- Creates heat maps of where obejects are
- Estimates how many objects are there
- [Descripition](https://towardsdatascience.com/objects-counting-by-estimating-a-density-map-with-convolutional-neural-networks-c01086f3b3ec)
- [Github](https://github.com/NeuroSYS-pl/objects_counting_dmap)

#### Other One implement later, maybe

- [Link](https://papers.nips.cc/paper/2010/file/fe73f687e5bc5280214e0486b273a5f9-Paper.pdf)

In [10]:
class FCRN(nn.Module):
    
    def __init__(self, N: int=1, input_filters: int=1, ):
        super(FCRN, self).__init__()
        self.model= nn.Sequential(
            # Downsampling
            conv_layer(input_filters, 32, (32, 32)),
            nn.MaxPool2d(2),
            
            conv_layer(32, 64, (32, 32)),
            nn.MaxPool2d(2),
            
            conv_layer(64, 128,(32, 32)),
            nn.MaxPool2d(2),
            
            # fully connected cnn
            conv_layer(128, 512, (32, 32)),
            
            # upsampling
            nn.Upsample(scale_factor=2),
            conv_layer(512, 128, (32, 32)),
            
            nn.Upsample(scale_factor=2),
            conv_layer(128, 64, (32, 32)),
            
            nn.Upsample(scale_factor=2),
            conv_layer(64, 1, (32, 32))
        )
        
    def forward(self, input: torch.Tensor):
        return self.model(input)



In [ ]:
# UMAP implementaion

In [11]:
fcrn = FCRN(input_filters=1)

# # Freeze pretrained model parameters to avoid backpropogating through them
# for parameter in fcrn.parameters():
#     parameter.requires_grad = False

from collections import OrderedDict
# New classifier 
classifier = nn.Sequential(OrderedDict([('fc1', nn.Linear(25088, 5000)),
                                        ('relu', nn.ReLU()),
                                        ('drop', nn.Dropout(p=0.5)),
                                        ('fc2', nn.Linear(5000, 6)),
                                        ('output', nn.LogSoftmax(dim=1))]))


# model.classifier = classifier
fcrn.classifier = classifier

criterion = nn.NLLLoss()
optimizer = optim.Adam(fcrn.classifier.parameters(), lr=0.001)

In [12]:
# Test Train Split
train_size = int(0.8*len(data))
test_size = len(data) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(data, [train_size, test_size])

In [13]:
# Create test and train dataloaders

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=4,
                                         shuffle=True, num_workers=2)

test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=4,
                                        shuffle=True, num_workers=2)

In [18]:
# Train CNN

def train_model(model, criterion, optimizer, train_loader):
    
    epochs = 2
    steps = 0
    print_every = 10
    model.to('cuda')
    
    for e in range(epochs):
        model.train()
        running_loss = 0
        
        for images, labels in iter(train_loader):
            steps +=1
            images, labels = images.to('cuda'), labels.to('cuda')
            optimizer.zero_grad()
            
            output = model.forward(images)
            loss = criterion(output, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            
            if steps % print_every == 0:
                
                model.eval()
                
                with torch.no_grad():
                        validation_loss, accuracy = validation(model, validate_loader, criterion)
            
                print("Epoch: {}/{}.. ".format(e+1, epochs+1),
                        "Training Loss: {:.3f}.. ".format(running_loss/print_every),
                        "Validation Loss: {:.3f}.. ".format(validation_loss/len(validate_loader)),
                        "Validation Accuracy: {:.3f}".format(accuracy/len(validate_loader)))
            
                running_loss = 0
                model.train()

In [25]:
it = iter(train_loader)
first = next(it)
second =next(it)

print(f' First: {type(first)}, Second: {type(second)}')

 First: <class 'list'>, Second: <class 'list'>


In [ ]:
train_model(fcrn, criterion, optimizer, train_loader)

In [15]:
# Test Model

def test_accuracy(model, test_loader):
    
    # Validate test set
    model.eval()
    model.to('cuda')
    
    with torch.no_grad():
        
        accuracy = 0
        
        for images, labels in iter(test_loader):
            
            images, labels = images.to('cuda'), labels.to('cuda')
            
            output = model.forward(images)
            
            probabilities = torch.exp(output)
            
            equality = (labels.data == probabilities.max(dim=1)[1])
            
            accuracy += equality.type(torch.FloatTensor).mean()
            
        print("Test Accuracy: {}".format(accuracy/len(test_loader)))

In [16]:
# Save/Load the checkpoint

def save_checkpoint(model):
    
    model.class_to_idx = training_dataset.class_to_idx
    
    checkpoint = {'arch':"vgg16",
                 'class_to_idx': model.class_to_idx,
                 'model_state_dict':model.state_dict()
                 }
    torch.save(checkpoint, 'checkpoint.pth')

def load_checkpoint(filepath):
    checkpoint = torch.load(filepath)
    
    if checkpoint['arch'] == 'vgg16':
        
        model = models.vgg16(pretrained=True)
        
        for param in model.parameters():
            param.requires_grad = False
        else:
            print('Architecture not recognized')
        
        model.class_to_idx = checkpoint['class_to_idx']
        classifier = nn.Sequential(OrderedDict([('fc1', nn.Linear(25088, 5000)),
                                            ('relu', nn.ReLU()),
                                            ('drop', nn.Dropout(p=0.5)),
                                            ('fc2', nn.Linear(5000, 102)),
                                            ('output', nn.LogSoftmax(dim=1))]))

    model.classifier = classifier
    
    model.load_state_dict(checkpoint['model_state_dict'])
    
    return model

In [ ]:
torch.cuda.empty_cache()